In [ ]:
# Python standard library imports
import time
from geopy.distance import geodesic

# Third-party imports for database connection and data manipulation
from sqlalchemy import create_engine
import pandas as pd

# Third-party imports for mapping
import folium
from folium.plugins import TimestampedGeoJson

Section 2: Connection

In [ ]:
# Database connection parameters
dbname = 'DataMining'
user = 'postgres'
password = 'datamining'
host = 'localhost'  # localhost or the server address
port = '5433'  # default PostgreSQL port is 5432

# Establish a connection to the database
connection_str = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"
engine = create_engine(connection_str)

The first function gets all the data for a certain veh_id sorted on timestamp
The second function checks for a dataframe that the concurrent entries are correct in the sense that they are very close location wise

In [ ]:
def fetch_data(veh_id):
    # Define the query with placeholders for parameters
    query = f"""
    SELECT * FROM vehicle_data 
    WHERE mapped_veh_id = {veh_id}
    ORDER BY timestamps_UTC;
    """

    # Start timing
    start_time = time.time()

    # Execute the query and fetch the data into a DataFrame
    df = pd.read_sql_query(query, engine)

    # End timing
    end_time = time.time()
    print(f"Query took {end_time - start_time} seconds to run.")
    # Close the database connection
    engine.dispose()
    
    return df

def entries_not_close(df, treshold_speed):
    """
    Check if consecutive entries in a DataFrame are not within a certain distance of each other
    and the time difference between them is less than a specified threshold. Also, calculate and 
    return the time difference, location difference, and speed (in km/h) between these entries.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing the location and time data.
    threshold_distance (float): The distance threshold in meters.
    time_threshold (str): The time difference threshold in Pandas timedelta string format.

    Returns:
    list of dicts: A list containing details of pairs of rows that are not close to each other 
                   and have a time difference less than the specified threshold.
    """
    not_close_entries = []

    for i in range(len(df) - 1):
        current_row = df.iloc[i]
        next_row = df.iloc[i + 1]

        # Calculate time difference in seconds
        time_diff = abs(pd.to_datetime(next_row['timestamps_utc']) - pd.to_datetime(current_row['timestamps_utc']))
        time_diff_seconds = time_diff.total_seconds()

        if time_diff_seconds <= 60:
            # Calculate the distance in meters
            distance = geodesic((current_row['lat'], current_row['lon']), (next_row['lat'], next_row['lon'])).meters
            
            speed_kmh = (distance / time_diff_seconds) * 3.6 if time_diff_seconds > 0 else 0

            # Check if the distance exceeds the threshold
            if speed_kmh > treshold_speed:
                # Calculate speed in km/h (meters per second to km/h conversion)

                # Append the result with details
                not_close_entries.append({
                    'indices': (i, i + 1),
                    'time_diff_seconds': time_diff_seconds,
                    'distance_meters': distance,
                    'speed_kmh': speed_kmh
                })

    return not_close_entries


In [ ]:
df = fetch_data(181)

In [ ]:
print(entries_not_close(df, 10000))

Here we observe that there are quite some entries where the speed can not be correct. After we checked on the map we saw that the points did not make sense. 

In [ ]:
# Index values identified by the function
index_pair = (221153, 221154)
print(df.iloc[index_pair[0]])
print(df.iloc[index_pair[1]])

In [ ]:
belgium_center = [50.5039, 4.4699]  # Roughly the center of Belgium
map_belgium = folium.Map(location=belgium_center, zoom_start=8)

folium.CircleMarker(
        location=[51.168687, 4.786373],
        radius=3,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(map_belgium)

folium.CircleMarker(
        location=[51.171492, 4.480429],
        radius=3,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(map_belgium)
map_belgium

Here for example the distance is more than 20km but the difference in time is 5 seconds.

In [ ]:
# Create a Folium map
belgium_center = [50.5039, 4.4699]  # Roughly the center of Belgium
m = folium.Map(location=belgium_center, zoom_start=12)

# Prepare data for the TimestampedGeoJson
features = []
count = 0
for idx, row in df.iterrows():
    count += 1
    print(count)
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row['lon'], row['lat']],
        },
        "properties": {
            "time": row['timestamps_utc'].isoformat(),
            "popup": f"Vehicle ID: {row['mapped_veh_id']}",
        },
    }
    features.append(feature)

# Add the TimestampedGeoJson to the map with a period of 10 minutes
TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": features,
    },
    period="PT2M",  # Change the time period to 10 minutes
    add_last_point=False,
    duration= "PT1M"
).add_to(m)
m

In [58]:
engine = create_engine(connection_str)
# Define the query with placeholders for parameters
query = f"""
SELECT * FROM vehicle_data 
WHERE (lat, lon) IN (
    SELECT lat, lon FROM vehicle_data 
    WHERE mapped_veh_id = 181
    GROUP BY lat, lon
    HAVING COUNT(*) > 1
)
AND mapped_veh_id = 181
ORDER BY timestamps_UTC;

"""

# Start timing
start_time = time.time()

# Execute the query and fetch the data into a DataFrame
df = pd.read_sql_query(query, engine)

# End timing
end_time = time.time()
print(f"Query took {end_time - start_time} seconds to run.")
#Close the database connection
engine.dispose()

Query took 13.496201038360596 seconds to run.


In [60]:
print(df)

     mapped_veh_id      timestamps_utc        lat       lon  \
0              181 2023-01-26 07:56:20  51.015416  3.775855   
1              181 2023-01-26 18:41:13  50.742513  3.603249   
2              181 2023-01-27 04:02:46  50.854848  3.605787   
3              181 2023-01-27 05:31:54  50.854848  3.605787   
4              181 2023-01-27 18:10:21  50.771254  3.618018   
..             ...                 ...        ...       ...   
498            181 2023-09-13 16:03:03  50.742557  3.603205   
499            181 2023-09-13 17:27:14  51.061597  3.739819   
500            181 2023-09-13 17:28:13  51.061597  3.739819   
501            181 2023-09-13 17:29:13  51.061597  3.739819   
502            181 2023-09-13 17:30:14  51.061597  3.739819   

     rs_e_inairtemp_pc1  rs_e_inairtemp_pc2  rs_e_oilpress_pc1  \
0                   0.0                 0.0                0.0   
1                  19.0                17.0              258.0   
2                  14.0                12.0  